DSC540 Term Project
Kurt Stoneburner

California COVID019 Ethnicity Data
https://data.ca.gov/dataset/covid-19-cases/resource/7e477adb-d7ab-4d4b-a198-dc4c6dc634c9

API Example: https://data.ca.gov/api/3/action/datastore_search?resource_id=7e477adb-d7ab-4d4b-a198-dc4c6dc634c9&limit=5

Requests Documentation: https://www.w3schools.com/python/ref_requests_response.asp

In [92]:
import requests
import json
import pandas as pd

In [85]:


#//*** Build Dictionary to hold Global values.
#//*** Placing Globals in a dictionary, keeps things tidy and helps with scope.
g = {
    #//*** Values for the API call
    "api" : {
        "url" : "https://data.ca.gov",
        "ethnic" : {
            "url" : "/api/3/action/datastore_search?resource_id=7e477adb-d7ab-4d4b-a198-dc4c6dc634c9",
            "colnames" : [], #//*** Column names
            #//*** Additional Column name attributes. Probably not needed. But ingesting anyway.
            #//*** Key - colname, value is attributes
            "attrib" : {}, 
        } #//*** CLOSE api.ethnic
        
    } #//*** CLOSE api

} #//***CLOSE g

#response = requests.get(g["requestCall"] + thisCityID + g["apiKey"])

#//*** Get the whole database 100 records at a time
response = requests.get(g['api']['url']+g["api"]["ethnic"]["url"])

print(response)

<Response [200]>


In [86]:
for x in response:
    print(x)

b'{"help": "https://data.ca.gov/api/3/action/help_show?name=datastore_search", "success": true, "result": {"include_total": true, '
b'"resource_id": "7e477adb-d7ab-4d4b-a198-dc4c6dc634c9", "fields": [{"type": "int", "id": "_id"}, {"info": {"notes": "Race/Ethnici'
b'ty", "type_override": "", "label": "Race Ethnicity"}, "type": "text", "id": "race_ethnicity"}, {"info": {"notes": "Cumulative nu'
b'mber of COVID confirmed cases as reported by local health departments", "type_override": "numeric", "label": "Total Positive Cas'
b'es"}, "type": "numeric", "id": "cases"}, {"info": {"notes": "Ratio of cases in relation to all cases", "type_override": "numeric'
b'", "label": "Percentage of Cases by Ethnicity"}, "type": "numeric", "id": "case_percentage"}, {"info": {"notes": "Cumulative num'
b'ber of COVID-related deaths as reported by local health departments", "type_override": "numeric", "label": "Death Percentage"}, '
b'"type": "numeric", "id": "deaths"}, {"info": {"notes": "Ratio of deaths in

In [87]:
print(response.ok)
#print(response.content)
#print(response.headers)
#print(response.json)
rawOBJ=json.loads(response.content)

print(rawOBJ["result"].keys())
for key, value in rawOBJ.items():
    print(f"{key} : {value}")
    

True
dict_keys(['include_total', 'resource_id', 'fields', 'records_format', 'records', 'limit', '_links', 'total'])
help : https://data.ca.gov/api/3/action/help_show?name=datastore_search
success : True
result : {'include_total': True, 'resource_id': '7e477adb-d7ab-4d4b-a198-dc4c6dc634c9', 'fields': [{'type': 'int', 'id': '_id'}, {'info': {'notes': 'Race/Ethnicity', 'type_override': '', 'label': 'Race Ethnicity'}, 'type': 'text', 'id': 'race_ethnicity'}, {'info': {'notes': 'Cumulative number of COVID confirmed cases as reported by local health departments', 'type_override': 'numeric', 'label': 'Total Positive Cases'}, 'type': 'numeric', 'id': 'cases'}, {'info': {'notes': 'Ratio of cases in relation to all cases', 'type_override': 'numeric', 'label': 'Percentage of Cases by Ethnicity'}, 'type': 'numeric', 'id': 'case_percentage'}, {'info': {'notes': 'Cumulative number of COVID-related deaths as reported by local health departments', 'type_override': 'numeric', 'label': 'Death Percentage

In [88]:

#//*** Build Column Names and Column attributes
rawFields = rawOBJ["result"]['fields']

#//*** Reset Columnnames in case this is run multiple times during debugging
g['api']['ethnic']['colnames'] = []

for loopOBJ in rawFields:
    #//*** Build attributes
    loopAttrib = {}
    
    #//*** All Columns have an info field except _id.
    if 'info' in loopOBJ.keys():
        loopAttrib = loopOBJ["info"]
    
    #//*** Add Type to loopAttrib
    loopAttrib['type'] = loopOBJ['type']
    
    #//*** The column name is the ID field.
    loopColname = loopOBJ['id']
    
    #//*** Add Column Name to the global api ethnic colnames 
    g['api']['ethnic']['colnames'].append(loopColname)
    
    #//*** Assign the attributes dictionary based on column name
    g['api']['ethnic']['attrib'][loopColname] = loopAttrib

#//**** Check Column names
print(g['api']['ethnic']['colnames'])

#//*** Check attributes
for x in g['api']['ethnic']['colnames']:
    print(g['api']['ethnic']['attrib'][x])

['_id', 'race_ethnicity', 'cases', 'case_percentage', 'deaths', 'death_percentage', 'percent_ca_population', 'date']
{'type': 'int'}
{'notes': 'Race/Ethnicity', 'type_override': '', 'label': 'Race Ethnicity', 'type': 'text'}
{'notes': 'Cumulative number of COVID confirmed cases as reported by local health departments', 'type_override': 'numeric', 'label': 'Total Positive Cases', 'type': 'numeric'}
{'notes': 'Ratio of cases in relation to all cases', 'type_override': 'numeric', 'label': 'Percentage of Cases by Ethnicity', 'type': 'numeric'}
{'notes': 'Cumulative number of COVID-related deaths as reported by local health departments', 'type_override': 'numeric', 'label': 'Death Percentage', 'type': 'numeric'}
{'notes': 'Ratio of deaths in relation to all deaths', 'type_override': 'numeric', 'label': 'Death Percentage', 'type': 'numeric'}
{'notes': 'Percent of State population', 'type_override': 'numeric', 'label': 'Percent of State Population', 'type': 'numeric'}
{'notes': 'Reported date

In [89]:
#//***print(f"{g['api']['ethnic']['colnames']}")
#//*** Build dictionary to hold raw data
rd = {}
#print(rawOBJ['result'].keys())

#//*** Use each column as a key, create and empty list for each column
for x in g['api']['ethnic']['colnames']:
    rd[x] = []

print(f"{g['api']['ethnic']['url']+nextCall}")
#print(rawOBJ['result'].keys())
    
#print(rawOBJ['result']['_links'])
#print(rawOBJ['result']['_links']['next'])
print(rawOBJ["success"])


while rawOBJ["success"]:
    #//*** Get Records as a List for each entry
    rawRecords = rawOBJ['result']["records"]
    print(f"Processing {len(rawRecords)}")

    #//*** Parse Each Record.
    for record in rawRecords:
        #//*** Each Record is an object.
        #//*** Each key is a column name.
        #//*** Loop through the Column names and append the value to the column stored in rd

        for col in g['api']['ethnic']['colnames']:
            #//*** Assign each element to the appropriate column
            rd[col].append(record[col])
    
    #//*** Check for ending while loop
    #//*** If the number of records returned is less than the limit, we are done
    if len(rawOBJ['result']["records"]) < rawOBJ['result']['limit']:
        print("Quitting Loop")
        break
    
    #//*** Check if there are more records to grab
    #//*** next contains the URL of the next request
    #//*** The API is limited to 100 records per API request.
    if 'next' in rawOBJ['result']['_links'].keys():
        ##//***API CODE HERE
        nextCall = rawOBJ['result']['_links']['next']
        
        #//*** Add the Next value to the base API call
        response = requests.get(g['api']['url']+nextCall)
        rawOBJ=json.loads(response.content)
        if rawOBJ["success"] == False:
            #//*** Break if Success returns False
            break
        
    else:
        #//*** Quit Here
        break
    



#print(rd)

#print(f"{rawOBJ['result']['records']}")

/api/3/action/datastore_search?resource_id=7e477adb-d7ab-4d4b-a198-dc4c6dc634c9/api/3/action/datastore_search?offset=3600&resource_id=7e477adb-d7ab-4d4b-a198-dc4c6dc634c9
True
Processing 100
100
Processing 100
200
Processing 100
300
Processing 100
400
Processing 100
500
Processing 100
600
Processing 100
700
Processing 100
800
Processing 100
900
Processing 100
1000
Processing 100
1100
Processing 100
1200
Processing 100
1300
Processing 100
1400
Processing 100
1500
Processing 100
1600
Processing 100
1700
Processing 100
1800
Processing 100
1900
Processing 100
2000
Processing 100
2100
Processing 4
Quitting Loop


In [94]:
    print(len(rd[g['api']['ethnic']['colnames'][0]]))
    
    #//*** API Data Stored in rd object.
    #//*** Build PANDAS DF
    
    ethnic_df = pd.DataFrame()
    
    for col in g['api']['ethnic']['colnames']:
        ethnic_df[col] = rd[col]
    
    print(f"{ethnic_df.head()}")

2104
   _id race_ethnicity  cases  case_percentage  deaths  death_percentage  \
0    1         Latino   5276            35.99     170             28.38   
1    2         Latino   5910            37.18     203             29.72   
2    3         Latino   6433            37.80     226             29.70   
3    4         Latino   7013            38.51     254             29.85   
4    5         Latino   7627            39.41     281             30.58   

   percent_ca_population                 date  
0                   38.9  2020-04-13T00:00:00  
1                   38.9  2020-04-14T00:00:00  
2                   38.9  2020-04-15T00:00:00  
3                   38.9  2020-04-16T00:00:00  
4                   38.9  2020-04-17T00:00:00  
